# IT41 Practical Session #3  - Starting with MPQP

---



In this practical session you will learn how to do and run your first quantum computation with MPQP

<table style="background: white;">
    <tr style="background: white;">
        <td> <img src='img/mpqp.png'  width="440px" />  
        <img src='img/colibri.png'  width="250px" />  
    </tr>
</table>

MPQP is an open-source framework for quantum computing​ based on Python and developped by the French start-up company ColibriTD (https://www.colibritd.com/). It allows to generate quantum circuits, run them on quantum simulators and/or on quantum computers for different provides (IBM, Azure, AWS, IonQ's, etc..).

All docs and info regarding the MPQP library can be found here: https://mpqpdoc.colibri-quantum.com/


---

## Import MPQP, the gates and the measurement tools

To run a quantum algorithm in the circuit formalism one needs to create a cricuit, to place gates and choose  measuement option. 
To do so, one first need to import the following libraries

```Python
from mpqp import* 
from mpqp.gates import*
from mpqp.measures import BasisMeasure
```

   

A quantum computation is often presented in the circuit formalism

<table style="background: white;">
    <tr style="background: white;">
        <td> <img src='img/circuit.png'  width="440px" />    
    </tr>
</table>



### Create a circuit
Thus the first thing to do is to create a circuit that will be made of quatum register (the wires that manipulates qubits) and classical registers (the wire where the values of the measurement will be recorded).

To create a circuit we will call the function QCircuit and specify the number of qubits $n$ one wants to use (here let's start with $n=1$):

```Python
circuit=QCirctuit(1)
```

### Vizualisation

To see our circuit one can use the following command:
```Python
print(circuit)
```



### Adding gates

Quantum operations are implemented by quantum gates, i.e. unitary matrices acting on the quantum state. Here are a couple of quantum gates implemented in MPQP: The Hadamard matrix, the NOT gate, the Z-gate

$$H=\frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1\\
1 & -1
\end{pmatrix},
X=\begin{pmatrix}
0 & 1\\
1 & 0
\end{pmatrix}, Z=\begin{pmatrix}
1 & 0\\
0 & -1
\end{pmatrix}
$$

To add a Hadamard gate to the circuit we can use the command:

```Python
circuit.add(H(0))
```
Note that the $H(0)$ indicates to which quantum register, the gate will be added to.
You can visualize the result by using again

```Python
print(circuit)
```

### Adding Measure operators

Once we have done as many operation on our circuit one can apply measurement operators. This is achieved by the command
```Python
circuit.add(BasisMEasure())
```

In the BasisMeasure command one can specify the qubit to be measured, the basis (standartd by default), the number of shots...

## Run the circuit

Once your circuit is properly generated you can run it on your own computer using a quantum simulator or on a real quantum device. The MPQP allows one to not only address different types of simulator but also different real hardware backend. One shows here some very simple possibilities offered by the multiplaform approach of MPQP 

First let's load the execution library

```Python
from mpqp.execution import*
```


### Run on AerSimulator (Qiskit Simulator) or Cirq (Google simulator)

The command to run a curcuit is 
```Python 
run() 
```

The command takes as entry the circuit to execute and the backend.

If one wants to run AerSimulator the local simulator of IBM or Cirq for Google, the sequence is

```Python
run(circuit,IBMDevice.AER_SIMULATOR)
```

Or


```Python
run(circuit,GOOGLEDevice.CIRQ_LOCAL_SIMULATOR)
```

It is also possible to use simulators that are desgined to respond with similar amount of error of the real backend. For instance to use the "Fake Brisbane' quantum computer one can run:

```Python
from mpqp.execution.simulated_devices import*
result=run(circuit,IBMSimulatedDevice.FakeBrisbane)
```

It is then possible to run on different devices at once:

```Python
run(circuit,[IBMDevice.AER_SIMULATOR,GOOGLEDevice.CIRQ_LOCAL_SIMULATOR,IBMSimulatedDevice.FakeBrisbane])
```

You can also gets the result by just printing the outcomes of the calculation
```Python
print(result)
```

###  Run on the IBM Quantum Computer

We explain here how to use MPQP to submit jobs on real Quantum hardware. 

First you should from the terminal run the command 

```Python
setup_connections
```

Then you just use the run command as we have seen before.

```Python
result=run(circuit,IBMDevice.IBM_LEAST_BUSY)
print(result)
```

Do not hesitate to check with the tabulation key to have acess to all different types of machines



## Two qubit circuit and entanglement

We have worked with a very simple circuit based on only one qubit manipulation. When one works with several qubit one can apply gates acting on several qubit. A very important gate is the CNOT gate or Controlled NOT gate. When apply on two qubit registers, the gate apply a NOT on the second register if the state of the first qubit is $|1>$.

The matrix of this 2-qubit gate is in the standard basis given by
$$CNOT=\begin{pmatrix}
1 & 0 &0 &0 \\
0 & 1 & 0 & 0\\
0 & 0 & 0  & 1\\
0 & 0 & 1  & 0
\end{pmatrix}$$

The symbol used for this gate is 
<table style="background: white;">
    <tr style="background: white;">
        <td> <img src='img/cnot.png'  width="150px" />    
    </tr>
</table>

The mpqp command to apply a CNOT gate should specify first the control qubit and then the target qubit:

```Python
circuit=QCircuit(2)
circuit.add(H(0))
circuit.add(CNOT(0,1))
```

Create the following circuit with MPQP   

<table style="background: white;">
    <tr style="background: white;">
        <td> <img src='img/epr.png'  width="440px" />    
    </tr>
</table>

And measure it in the standard basis with the simulator and a quantum machine.

MPQP also allows to directly to provide the circuit in a sequential array. For instance the previous circuit can be written as
```Python
circuit=QCirctuit([H(0),CNOT(0,1),BasisMeasure([0,1],shots=1000)])
```

## Measuring a state in an alternative basis (optional)

The command 'BasisMeasure' only allows to make single qubit measurement in the standard basis $|0>,|1>$. MPQP allows one to create our own basis of measurements

However it is also possible to make a measurement in any basis $|u_1>,|u_2>$ by making a rotation before the measurement.

Suppose the unitary matrix $U$ is the change of basis from $|u_1>, |u_2>$ to the standard basis $|0>,|1>$. Then applying $U$ and a measurement in the the $Z$-basis is equivalent to making a measurment in the $|u_1>,|u_2>$ basis.

For example for a measurement in the $X$-basis of the quantum register $i$ we first apply the rotation given by the Hadamard matrix:
```Python
circuit.add(H(i),BasisMeasure([i]))
```

For measuring in the $Y$-basis ($\frac{|0>+i|1>}{\sqrt{2}}, \frac{|0>-i|1>}{\sqrt{2}}$ the change of basis is obtained by applying the matrices $S^\dagger=\begin{pmatrix}
1 & 0\\
0 & e^{-i\pi/2}
\end{pmatrix}$ and $H$:

```Python
circuit.add([Rk_dagger(2,i),H(i),BasisMeasure([i])])
```


More generally the rotation gate U allows one to make a change of basis before measurements

## Exercises

### Exercise 1 (super dense conding)

The super dense coding protocol is a communication protocol that allows to transmit two classical bit by sending only one qubit. It was created in Bennet and Wiesner.
The following circuit represents the protocol:

<table style="background: white;">
    <tr style="background: white;">
        <td> <img src='img/superdense.png'  width="440px" />    
    </tr>
</table>

Alice and Bob have each of them one qubit of an entangled EPR states
$$|EPR>=\dfrac{1}{\sqrt{2}}(|00>+|11>)$$
Then if Alice wants to send the two classical bits $(b_1,b_2)$ with $b_1, b_2 \in\{0,1\}$, she first applies $X^{b_2}$ then $Z^{b_1}$ to her qubit and send it to Bob (if $b_i=0$ she applies the identity matrix). Then Bob applies the CNOT gate and the Hadamard gate on the first register and measures the two qubit.

Implement the protocol and check that in all four cases, Alice is transmetted two classical bits by only sending one qubit.

### Exercise 2: Quantum Teleportation

The quantum teleportation protocol allows to transfer the information (amplitudes) defining a qubit $|\psi>$ to another qubit by taking advantage of entanglement.

Here is the circuit corresponding to the protocol
<table style="background: white;">
    <tr style="background: white;">
        <td> <img src='img/teleportation2.png'  width="440px" />    
    </tr>
</table>

This protocol insures that the qubit $|q_1>$ on the quantum register $0$ will be in the quantum register $2$ ($|q_3>$) at the end of the protocol.


1) Prepare the qubit $|q_1>$ such that $|q_1>=|1>$ and check that the measurement of the last qubit gives $1$ with probability $1$.

2) Same question with $|q_1>=|+>$. How do you check that the last qubit in in state $|+>$ at the end of the procotol ?

3) (more difficult) Can you preparer any state $|q_1>=|\psi>$ and provide some argument to check that the qubit $|\psi>$ has been teleported at the end of the procotol ? (you may need to use the $U_3$ gate of qiskit and read the optional section on "Measurement in an alternative basis"

